In [1]:
from heapq import heappush, heappop

cost_grid = [
    [1, 1, 2, None, 1],
    [1, None, 2, None, 2],
    [1, 1, 1, 1, 2],
    [2, None, None, 1, 1],
    [1, 1, 1, None, 1]
]

start = (0, 0)
goal = (4, 4)

def in_bounds(cell):
    r, c = cell
    return 0 <= r < len(cost_grid) and 0 <= c < len(cost_grid[0])

def passable(cell):
    r, c = cell
    return cost_grid[r][c] is not None

def neighbors(cell):
    r, c = cell
    for dr, dc in ((1, 0), (-1, 0), (0, 1), (0, -1)):
        nxt = (r + dr, c + dc)
        if in_bounds(nxt) and passable(nxt):
            yield nxt

In [2]:
def manhattan(a, b):
    """Heuristic: distance if we can only move in straight lines."""
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

In [3]:
def a_star(grid, start, goal):
    """Classic A* search that combines real cost (g) and heuristic (h)."""
    frontier = []
    heappush(frontier, (manhattan(start, goal), 0, start))  

    came_from = {start: None}
    g_score = {start: 0}

    while frontier:
        f, current_cost, current = heappop(frontier)
        if current == goal:
            break

        for nxt in neighbors(current):
            step_cost = grid[nxt[0]][nxt[1]]
            tentative_g = current_cost + step_cost

            if tentative_g < g_score.get(nxt, float("inf")):
                came_from[nxt] = current
                g_score[nxt] = tentative_g
                new_f = tentative_g + manhattan(nxt, goal)
                heappush(frontier, (new_f, tentative_g, nxt))

    return came_from, g_score

In [4]:
def build_path(came_from, start, goal):
    if goal not in came_from:
        return []
    path = []
    current = goal
    while current is not None:
        path.append(current)
        current = came_from[current]
    path.reverse()
    return path

In [5]:
came_from, g_scores = a_star(cost_grid, start, goal)
path = build_path(came_from, start, goal)

if path:
    total_cost = g_scores[goal]
    print("Optimal path coordinates:")
    for step, cell in enumerate(path, start=1):
        print(f"{step}. {cell}")
    print("\nTotal path cost:", total_cost)
else:
    print("No path found.")

visual_grid = []
for r, row in enumerate(cost_grid):
    visual_row = []
    for c, value in enumerate(row):
        if (r, c) == start:
            visual_row.append("S")
        elif (r, c) == goal:
            visual_row.append("G")
        elif (r, c) in path:
            visual_row.append("*")
        elif value is None:
            visual_row.append("#")
        else:
            visual_row.append(str(value))
    visual_grid.append(visual_row)

print("\nGrid view (S=start, G=goal, *=path, #=wall, numbers=cost):")
for row in visual_grid:
    print(" ".join(row))

Optimal path coordinates:
1. (0, 0)
2. (1, 0)
3. (2, 0)
4. (2, 1)
5. (2, 2)
6. (2, 3)
7. (3, 3)
8. (3, 4)
9. (4, 4)

Total path cost: 8

Grid view (S=start, G=goal, *=path, #=wall, numbers=cost):
S 1 2 # 1
* # 2 # 2
* * * * 2
2 # # * *
1 1 1 # G


Good afternoon, ma’am. This program is based on the A-star search algorithm, which is one of the most popular and efficient algorithms for finding the shortest and optimal path between two points on a grid. It combines both the actual travel cost and a heuristic estimate of the remaining distance, which makes it faster and smarter than algorithms like Breadth-First Search or Greedy Best-First Search.

At the beginning, I import two functions, heappush and heappop, from Python’s heapq module. These functions help me create a priority queue, where the smallest cost or priority item is always selected first. This queue is used to store the cells to be explored next.

Next, I define a grid called cost_grid. It’s a two-dimensional list that represents our map or area. Each number in the grid shows the cost to move through that cell — smaller numbers mean cheaper or easier to move, and None means that the cell is blocked or has a wall, so we cannot pass through it. The grid has five rows and five columns.

I then define two tuples: start = (0, 0) and goal = (4, 4). These represent the coordinates of the starting position and the goal position in the grid. The start is at the top-left corner, and the goal is at the bottom-right corner.

After that, I define a helper function called in_bounds(cell). This function checks whether a given cell is within the valid boundaries of the grid. It takes the row and column numbers from the cell and ensures they are between 0 and the grid size limits. If the cell is outside, it returns False.

The next helper function is passable(cell). This checks if a cell is not a wall. It looks at the grid value at that cell, and if the value is None, it means that the cell is blocked. Otherwise, it’s passable.

Then I define the neighbors(cell) function, which is used to find all valid neighboring cells around the current cell. We can move in four directions — down, up, right, and left. For each direction, it creates a new cell by adding or subtracting 1 from the row or column. It then checks if the new cell is inside the grid using in_bounds() and also checks if it’s passable using passable(). If both conditions are true, that neighbor is valid, and it is yielded as a possible next move.

Next, I define a function manhattan(a, b). This function is our heuristic function — it estimates how far a cell a is from cell b if we can only move in straight lines, not diagonally. It calculates the Manhattan distance using the formula abs(a[0] - b[0]) + abs(a[1] - b[1]). This value helps guide the search toward the goal.

Now comes the main function, a_star(grid, start, goal). It performs the actual A-star search. Inside it, I first create an empty list called frontier, which will act as a priority queue. Then I push the starting node into the queue using heappush(frontier, (manhattan(start, goal), 0, start)). Here, the first value is f = h + g, which represents the estimated total cost (heuristic plus actual cost), the second value 0 is the actual cost from the start, and the third value is the position itself.

Next, I create a dictionary called came_from, which keeps track of each cell’s parent — meaning, from which cell we reached it. I set the start cell’s parent as None. I also create another dictionary called g_score that stores the actual cost from the start to each cell, and for the start cell, that cost is 0.

Then, I start a while loop that runs until the frontier is empty. Inside it, I remove the cell with the smallest total estimated cost using heappop(frontier). This gives me three values: f, which is the total estimated cost; current_cost, which is the cost so far; and current, which is the current cell we are exploring.

If the current cell is equal to the goal, that means we have reached the destination, so we break out of the loop because we’ve found the shortest path.

If not, we go through all valid neighbors of the current cell using a for loop. For each neighbor, we get the movement cost from the grid and calculate a tentative cost called tentative_g, which is the cost so far plus the cost to move into that neighbor cell.

If this new cost is less than the previously known cost stored in g_score, it means we’ve found a cheaper route to this neighbor. So, we update its parent in came_from, update its cost in g_score, and then calculate a new total estimated cost new_f = tentative_g + manhattan(nxt, goal). Finally, we push this neighbor into the priority queue using heappush.

After the loop finishes, the function returns both came_from and g_score, which store all the information about how we reached each cell and the cost to reach it.

Next, I define another function called build_path(came_from, start, goal). This function reconstructs the path from the start to the goal by following the parent links backward. We start from the goal and keep moving to its parent until we reach the start. We store all these cells in a list called path, then reverse it so it shows the path from start to goal.

After defining all the functions, I call a_star(cost_grid, start, goal) to perform the search and store the results in came_from and g_scores. Then, I call build_path(came_from, start, goal) to reconstruct the final shortest path.

If the path is not empty, that means a route was found. I calculate the total cost from g_scores[goal] and print all the coordinates of the path step by step with their numbers. Then I print the total cost of the path.

If the path is empty, that means there’s no possible route to reach the goal, and I print “No path found.”

Next, I create a visual representation of the grid. I loop through each cell, and for each one, I display different symbols:

S for the start cell

G for the goal cell

* for the cells that form the path

# for blocked or wall cells (where value is None)

And the numbers themselves for the cost values of open cells.

Finally, I print the grid clearly so we can visually see the path from S to G marked with stars. This helps understand how the algorithm worked in the maze.

In summary, this program uses the A-star algorithm to find the optimal and lowest-cost path between two points on a grid. It combines the actual travel cost and the estimated remaining cost using the Manhattan heuristic. As a result, it is faster and more accurate than simple search methods, and it is commonly used in robot navigation, games, and map routing applications.